# BARPA-R Future Tropical Low Case Study

In a warmer climate, moist thermodynamics imply that the atmosphere will be able to carry more moisture.

This means that rainfall events have the potential to become more intense in the future.

While CMIP6 models are able to simulate the increasing levels of atmospheric moisture, downscaled projections are needed to see the effect on small-scale weather systems that global models do not resolve.

This case-study looks at BARPA-R downscaling of EC-Earth3. Of all models downscaled with BARPA-R, EC-Earth3 shows the largest increase in atmospheric moisture.

We have identified two tropical low events from BARPA-R-EC-Earth3 with similar tracks paths and minimum pressures. 

Follow the tutorial to see how the rainfall and atmospheric moisture vary between the two lows.


## Setup Notebook

In [ ]:
%matplotlib notebook
# this is needed for animations

In [ ]:
# First lets import some libraries
import os, sys
os.chdir("/g/data/tp28/dev/eh6215/barpa-barra2-amos2024/")
import loaddata
from glob import glob
import xarray as xr
import pandas as pd
import cartopy.crs as ccrs
from dask.distributed import Client, LocalCluster
from matplotlib import pyplot as plt
from datetime import datetime 
from  matplotlib import cm

In [ ]:
# Set up Dask
cluster = LocalCluster()
client = Client(cluster)
client

In [ ]:
# In order to highlight the rich data available from BARPA, this tutorial features animations
# The following settings, as well as "%matplotlib notebook" at the beginning of the note-book, allow these animations to run


from matplotlib.widgets import Slider
import matplotlib.animation
plt.rcParams["animation.html"] = "jshtml"
plt.rcParams['figure.dpi'] = 150  
#plt.rcParams["animation.embed_limit"] = 60   # set this to a number larger than the default (20 MB) to generate larger animations
plt.ioff()

## Load data

In [ ]:
# Now, let's load our first tropical low
# Remember this is a synthetic, model-generated track, it didn't occur in the real world!
# the variables we're loading are:
#   psl: sea level pressure
#   pr: precipitation
#   prw: atmospheric total column water vapour
dict_hist_vars = {}

# dict_hist_vars # historical
# dict_proj_vars # projections
for var in ['psl','pr','prw']:
    dict_hist_vars[var] = loaddata.load_barpa_data("BARPA-R",
                         "EC-Earth3",
                         "historical",
                         "1hr",
                         var,
                         tstart = datetime(2009,1,5),
                         tend=datetime(2009,1,16),
                         lonrange = (105,160),
                         latrange = (-45,-7))[var]
    
    
# Now, let's change the precipitation units into something more understandable
dict_hist_vars['pr'][:] = dict_hist_vars['pr'][:]*3600
dict_hist_vars['pr'].attrs['units']='mm hr-1'

In [ ]:
# let's check these are what I say they are, and their units:
for var in dict_hist_vars: 
    print(f"{var:3s}: {dict_hist_vars[var].long_name:25s} ({dict_hist_vars[var].attrs['units']})")


In [ ]:
# Now we load the second low

dict_proj_vars = {}
for var in ['psl','pr','prw']:
    dict_proj_vars[var] = loaddata.load_barpa_data("BARPA-R",
                         "EC-Earth3",
                         "ssp370",
                         "1hr",
                         var,
                         tstart = datetime(2088,12,25),
                         tend=datetime(2089,1,6),
                         lonrange = (105,160),
                         latrange = (-45,-7))[var]
    
# Now, let's change the precipitation units into something more understandable
dict_proj_vars['pr'][:] = dict_proj_vars['pr'][:]*3600
dict_proj_vars['pr'].attrs['units']='mm hr-1'


## Preprocess data

In [ ]:
# to start with, we'll look at 6-hourly data. You can increase the frequency later (by rerunning the whole notebook) if you'd lke

for var in dict_hist_vars:
    dict_hist_vars[var] = dict_hist_vars[var].resample(time='6h').mean()
    dict_proj_vars[var] = dict_proj_vars[var].resample(time='6h').mean()

#### Now let's find those lows!


In this example, the systems we're interested in are the lowest-pressure 
objects in the box 100 E-150 E, 30S - 0S. 
So we can just find the minimum over space at each timestep to create the track paths

In [ ]:
track_longitude, track_latitude = [0,0],[0,0]
for i,da in enumerate([dict_hist_vars,dict_proj_vars]):
    # subset to a box surrounding the lows
    subsetted_pressure = da['psl'].sel(lon=slice(100,150),lat=slice(-30,0))
    # extract latitude and longitudes
    subsetted_lon = subsetted_pressure.lon
    subsetted_lat = subsetted_pressure.lat
    # find location of minimum pressure at each time
    track_xindex = subsetted_pressure.min('lat').argmin('lon')
    track_yindex = subsetted_pressure.min('lon').argmin('lat')
    # find coordinates of minimum pressures
    track_longitude[i] = subsetted_lon[track_xindex]
    track_latitude[i] = subsetted_lat[track_yindex]

To start off, we'll create an animation of total column water, with precipitation and the track path drawn on top.
Precip will be masked at rates of <2mm/hour so the prw is visible underneath. 


## Animation

In [ ]:

# to create an animation we first create the initial frame. 

# set up the figure
fig=plt.figure(figsize=(12,6)) # set up figure

# empty objects for the animated components
ax,pcolor1,pcolor2,dot = [0,0],[0,0],[0,0],[0,0]

# populate the initial frame
for i,da in enumerate([dict_hist_vars,dict_proj_vars]):
    ax[i]=plt.subplot(1,2,i+1,projection=ccrs.PlateCarree()) #set up axes with geometric projection
    # plot initial hourly precip rate, with sec-> hour conversion rate
    pcolor1[i] = (da['prw'][0]).plot(vmin=0,
                                     vmax=100,
                                     cmap=cm.get_cmap('Blues',10),
                                     add_colorbar=False) 
    # plot initial hourly precip rate, with sec-> hour conversion rate
    pcolor2[i] = (da['pr'][0].where(da['pr'][0]>2)).plot(vmin=0,
                                                         vmax=50,
                                                         cmap=cm.get_cmap('turbo',10),
                                                         add_colorbar=False) 
    # draw coastlines
    ax[i].coastlines() 
    ax[i].set_xlim(105,160)
    ax[i].set_ylim(-45,-7)
    # set timestamp as axes label
    ax[i].set_title(da['prw'].time[0].values.__str__()[:16]) 
    # plot full cyclone track
    ax[i].plot(track_longitude[i][:-1],track_latitude[i][:-1],c='magenta') 
    # plot initial cyclone location
    dot[i] = plt.plot(track_longitude[0],track_latitude[0],'o',c='magenta',ms=4) 

    
fig.subplots_adjust(top=0.95, bottom=0.3)
cax1 = fig.add_axes([0.3,0.1,0.4,0.05])
fig.colorbar(pcolor1[i],
             cax=cax1,
             orientation='horizontal')
cax1.set_xlabel("Total Column Water, kg/m2")
cax2 = fig.add_axes([0.3,0.25,0.4,0.05])
fig.colorbar(pcolor2[i],
             cax=cax2,
             orientation='horizontal')
cax2.set_xlabel("Precipitation, mm/hr")
#fig.tight_layout()

## todo

# second, we create a function that will update the figure to time-step t
def animate(t): 
    for i,da in enumerate([dict_hist_vars,dict_proj_vars]):
        # update prw to time t
        pcolor1[i].set_array(da['prw'][t].values.ravel()) 
        # update rainfall to time t
        pcolor2[i].set_array(da['pr'][t].where(da['pr'][t]>2).values.ravel()) 
        # update cyclone lon to time t
        dot[i][0].set_xdata([track_longitude[i][t]]) 
        dot[i][0].set_ydata([track_latitude[i][t]]) 
        # update cyclone lat to time t
        ax[i].set_title(da['prw'].time[t].values.__str__()[:16]) # update title

# finally, we use FuncAnimation to generate our figure
timesteps = min(len(dict_hist_vars['pr'].time), len(dict_proj_vars['pr'].time))
matplotlib.animation.FuncAnimation(fig, animate, frames=timesteps)  # generate animation


# this may take a while!

Congrats, you've completed the first runthrough of the notebook. Now, get creative. You might like to:

Simple extensions:
* Increase the update frequency from 6 hours 
* Look at some other variables: consider
    * maximum wind gust (wsgsmax)
    * Temperature (tas)
    * Clouds (cll/clm/clt, or rlut is a good proxy)
    * 6-hourly Winds on pressure levels (uaXXX, vaXXX, where XXX is the pressure level)
* Generate some still figures that summarise your key findings from this example
 
More complicated extensions:
* Try to add wind barbs, contours or streamfunctions
* Adapt to look at the 3D structure of the system
* Compute dewpoint temperatures, or moist static energy

In [ ]:
# A hint:
# Jupyter currently doesn't allow both animations and still figures to be displayed in the same notebook
# the following work around is needed to display stills:


fig = plt.figure(figsize=(3,3))
plt.plot([1,2,3],[4,5,6],'.')

def animate(t):
    return
    
matplotlib.animation.FuncAnimation(fig, animate, frames=1)  # generate animation

# alternatively, copy to a new notebook and remove the line
# %matplotlib notebook